# 🤖 Festeja Kids - Integração com APIs de IA

**Notebook para testar e usar APIs de IA no Google Colab**

Este notebook demonstra como integrar diferentes APIs de IA para automatizar tarefas do Festeja Kids.

---

## 🔐 Configuração de API Keys

**IMPORTANTE**: Nunca exponha suas chaves API no código!

### Como adicionar chaves de forma segura no Colab:

1. Clique no ícone 🔑 (chave) na barra lateral esquerda
2. Clique em **+ Add new secret**
3. Adicione cada chave:
   - `MANUS_API_KEY` = `FESTEJA-KIDS2-API-241120250201`
   - `OPENAI_API_KEY` = sua chave OpenAI
   - `GEMINI_API_KEY` = sua chave Google Gemini
   - etc.

---

## 📦 Instalação de Bibliotecas

In [ ]:
# Instalar todas as bibliotecas necessárias
!pip install -q openai google-generativeai anthropic requests

## 🔑 Carregar API Keys

In [ ]:
from google.colab import userdata
import os

# Carregar chaves de forma segura
try:
    MANUS_API_KEY = userdata.get('MANUS_API_KEY')
    print("✅ Manus API Key carregada")
except:
    print("⚠️ Manus API Key não encontrada. Adicione em Secrets.")
    MANUS_API_KEY = None

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API Key carregada")
except:
    print("⚠️ OpenAI API Key não encontrada. Adicione em Secrets.")
    OPENAI_API_KEY = None

try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ Gemini API Key carregada")
except:
    print("⚠️ Gemini API Key não encontrada. Adicione em Secrets.")
    GEMINI_API_KEY = None

print("\n✅ Configuração concluída!")

## 🎯 1. API Manus (Já Configurada)

In [ ]:
import requests
import json

def chamar_manus_llm(mensagem, sistema="Você é um assistente de festas infantis."):
    """
    Chama a API Manus LLM
    """
    if not MANUS_API_KEY:
        return "❌ API Key do Manus não configurada"
    
    url = "https://forge.manus.im/llm/chat"
    headers = {
        "Authorization": f"Bearer {MANUS_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "messages": [
            {"role": "system", "content": sistema},
            {"role": "user", "content": mensagem}
        ]
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        return f"❌ Erro: {str(e)}"

# Teste
print("🎉 Testando API Manus...\n")
resposta = chamar_manus_llm("Sugira 5 temas de festa para crianças de 5 anos")
print(resposta)

## 🤖 2. OpenAI API (GPT-4 / GPT-3.5)

In [ ]:
from openai import OpenAI

def chamar_openai(mensagem, modelo="gpt-3.5-turbo", sistema="Você é um especialista em festas infantis."):
    """
    Chama a API OpenAI
    """
    if not OPENAI_API_KEY:
        return "❌ API Key da OpenAI não configurada"
    
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    try:
        response = client.chat.completions.create(
            model=modelo,
            messages=[
                {"role": "system", "content": sistema},
                {"role": "user", "content": mensagem}
            ],
            temperature=0.7,
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ Erro: {str(e)}"

# Teste
print("🤖 Testando OpenAI...\n")
resposta = chamar_openai("Liste 10 atividades divertidas para festa de Frozen")
print(resposta)

## 🌟 3. Google Gemini API

In [ ]:
import google.generativeai as genai

def chamar_gemini(mensagem):
    """
    Chama a API Google Gemini
    """
    if not GEMINI_API_KEY:
        return "❌ API Key do Gemini não configurada"
    
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    
    try:
        response = model.generate_content(mensagem)
        return response.text
    except Exception as e:
        return f"❌ Erro: {str(e)}"

# Teste
print("🌟 Testando Google Gemini...\n")
resposta = chamar_gemini("Crie um cardápio saudável para festa infantil com 50 crianças")
print(resposta)

## 🎯 Casos de Uso Práticos

### 📝 Caso 1: Gerador de Descrições de Festas

In [ ]:
def gerar_descricao_festa(tema, idade, convidados, api="manus"):
    """
    Gera descrição de marketing para uma festa
    """
    prompt = f"""
    Crie uma descrição de marketing atrativa e profissional para uma festa infantil com:
    
    - Tema: {tema}
    - Idade da criança: {idade} anos
    - Número de convidados: {convidados}
    
    A descrição deve:
    1. Ter 2-3 parágrafos
    2. Destacar a diversão e segurança
    3. Mencionar atividades típicas do tema
    4. Ser envolvente e profissional
    """
    
    if api == "manus":
        return chamar_manus_llm(prompt)
    elif api == "openai":
        return chamar_openai(prompt)
    elif api == "gemini":
        return chamar_gemini(prompt)

# Teste
print("=" * 60)
print("📝 GERADOR DE DESCRIÇÕES DE FESTAS")
print("=" * 60)

descricao = gerar_descricao_festa(
    tema="Patrulha Canina",
    idade=4,
    convidados=30,
    api="manus"  # Trocar para "openai" ou "gemini" se preferir
)

print(f"\n{descricao}\n")
print("=" * 60)

### 💡 Caso 2: Sugestão de Atividades

In [ ]:
def sugerir_atividades(tema, idade, duracao_horas=3):
    """
    Sugere atividades para a festa
    """
    prompt = f"""
    Sugira um cronograma de atividades para uma festa infantil de {tema} para crianças de {idade} anos.
    
    A festa dura {duracao_horas} horas.
    
    Para cada atividade, forneça:
    1. Nome da atividade
    2. Duração estimada
    3. Materiais necessários
    4. Breve descrição
    
    Organize em ordem cronológica.
    """
    
    return chamar_manus_llm(prompt)

# Teste
print("=" * 60)
print("💡 SUGESTÃO DE ATIVIDADES")
print("=" * 60)

atividades = sugerir_atividades(
    tema="Super-Heróis",
    idade=6,
    duracao_horas=3
)

print(f"\n{atividades}\n")
print("=" * 60)

### 💰 Caso 3: Gerador de Orçamento

In [ ]:
def gerar_orcamento(tema, convidados, orcamento_total):
    """
    Gera orçamento detalhado para a festa
    """
    prompt = f"""
    Crie um orçamento detalhado para uma festa infantil com:
    
    - Tema: {tema}
    - Número de convidados: {convidados}
    - Orçamento total: R$ {orcamento_total:,.2f}
    
    Distribua o orçamento nas seguintes categorias:
    1. Decoração (30%)
    2. Alimentação (35%)
    3. Entretenimento (20%)
    4. Lembrancinhas (10%)
    5. Outros (5%)
    
    Para cada categoria:
    - Valor alocado
    - Lista de itens específicos
    - Quantidade estimada
    - Custo unitário aproximado
    
    Formate como uma tabela clara.
    """
    
    return chamar_manus_llm(prompt)

# Teste
print("=" * 60)
print("💰 GERADOR DE ORÇAMENTO")
print("=" * 60)

orcamento = gerar_orcamento(
    tema="Frozen",
    convidados=50,
    orcamento_total=5000.00
)

print(f"\n{orcamento}\n")
print("=" * 60)

### 📊 Caso 4: Análise de Feedback

In [ ]:
import json

def analisar_feedback(texto_feedback):
    """
    Analisa sentimento e extrai insights de feedback
    """
    prompt = f"""
    Analise o seguinte feedback de uma festa infantil:
    
    "{texto_feedback}"
    
    Retorne APENAS um JSON válido com:
    {{
        "sentimento": "positivo/neutro/negativo",
        "score": 0-10,
        "pontos_fortes": ["lista de pontos positivos"],
        "pontos_fracos": ["lista de pontos negativos"],
        "acao_recomendada": "o que fazer com base no feedback"
    }}
    """
    
    resposta = chamar_manus_llm(prompt)
    
    try:
        # Tentar extrair JSON da resposta
        inicio = resposta.find('{')
        fim = resposta.rfind('}') + 1
        json_str = resposta[inicio:fim]
        return json.loads(json_str)
    except:
        return {"erro": "Não foi possível parsear JSON", "resposta_bruta": resposta}

# Teste
print("=" * 60)
print("📊 ANÁLISE DE FEEDBACK")
print("=" * 60)

feedback_exemplo = """
A festa foi maravilhosa! As crianças adoraram a decoração do tema Frozen.
A comida estava deliciosa e chegou na hora certa. A única coisa que poderia
melhorar seria ter mais opções de brincadeiras ao ar livre, pois estava um
dia lindo e as crianças queriam brincar fora. No geral, super recomendo!
"""

analise = analisar_feedback(feedback_exemplo)
print(f"\n{json.dumps(analise, indent=2, ensure_ascii=False)}\n")
print("=" * 60)

### 🎨 Caso 5: Gerador de Paleta de Cores

In [ ]:
def gerar_paleta_cores(tema):
    """
    Gera paleta de cores para decoração
    """
    prompt = f"""
    Sugira uma paleta de cores para decoração de festa infantil com tema {tema}.
    
    Forneça:
    1. Cor principal (com código hexadecimal)
    2. Cor secundária (com código hexadecimal)
    3. Cor de destaque/acento (com código hexadecimal)
    4. Cor neutra (com código hexadecimal)
    5. Sugestões de como usar cada cor na decoração
    
    Formato:
    Cor Principal: [nome] (#XXXXXX) - [uso]
    """
    
    return chamar_manus_llm(prompt)

# Teste
print("=" * 60)
print("🎨 GERADOR DE PALETA DE CORES")
print("=" * 60)

paleta = gerar_paleta_cores("Unicórnio")
print(f"\n{paleta}\n")
print("=" * 60)

### 💬 Caso 6: Chatbot de Atendimento

In [ ]:
def chatbot_atendimento(mensagem, historico=[]):
    """
    Simula chatbot de atendimento
    """
    sistema = """
    Você é um atendente virtual da Festeja Kids, empresa de festas infantis.
    
    Seja cordial, prestativo e profissional.
    
    Seu objetivo é coletar as seguintes informações:
    1. Nome do cliente
    2. Telefone
    3. Data desejada para a festa
    4. Tema preferido
    5. Número de convidados
    6. Orçamento aproximado
    
    Faça perguntas uma de cada vez.
    Quando tiver todas as informações, resuma e pergunte se pode agendar uma visita.
    """
    
    # Construir histórico para contexto
    mensagens_contexto = "\n".join([
        f"{msg['role']}: {msg['content']}" 
        for msg in historico
    ])
    
    prompt_completo = f"""
    Histórico da conversa:
    {mensagens_contexto}
    
    Cliente: {mensagem}
    
    Responda como atendente:
    """
    
    resposta = chamar_manus_llm(prompt_completo, sistema=sistema)
    
    # Atualizar histórico
    historico.append({"role": "cliente", "content": mensagem})
    historico.append({"role": "atendente", "content": resposta})
    
    return resposta, historico

# Teste - Simulação de conversa
print("=" * 60)
print("💬 CHATBOT DE ATENDIMENTO")
print("=" * 60)

historico = []

# Mensagem 1
print("\n👤 Cliente: Olá, gostaria de fazer uma festa para minha filha\n")
resposta, historico = chatbot_atendimento(
    "Olá, gostaria de fazer uma festa para minha filha",
    historico
)
print(f"🤖 Atendente: {resposta}\n")

# Mensagem 2
print("👤 Cliente: Meu nome é Maria Silva\n")
resposta, historico = chatbot_atendimento(
    "Meu nome é Maria Silva",
    historico
)
print(f"🤖 Atendente: {resposta}\n")

# Mensagem 3
print("👤 Cliente: (11) 98765-4321\n")
resposta, historico = chatbot_atendimento(
    "(11) 98765-4321",
    historico
)
print(f"🤖 Atendente: {resposta}\n")

print("=" * 60)

## 📊 Comparação de APIs

In [ ]:
import time

def comparar_apis(prompt):
    """
    Compara velocidade e qualidade das APIs
    """
    print("=" * 60)
    print("📊 COMPARAÇÃO DE APIs")
    print("=" * 60)
    print(f"\nPrompt: {prompt}\n")
    
    # Manus
    if MANUS_API_KEY:
        print("\n🎯 MANUS:")
        inicio = time.time()
        resposta = chamar_manus_llm(prompt)
        tempo = time.time() - inicio
        print(f"⏱️ Tempo: {tempo:.2f}s")
        print(f"📝 Resposta: {resposta[:200]}...\n")
    
    # OpenAI
    if OPENAI_API_KEY:
        print("\n🤖 OPENAI (GPT-3.5):")
        inicio = time.time()
        resposta = chamar_openai(prompt, modelo="gpt-3.5-turbo")
        tempo = time.time() - inicio
        print(f"⏱️ Tempo: {tempo:.2f}s")
        print(f"📝 Resposta: {resposta[:200]}...\n")
    
    # Gemini
    if GEMINI_API_KEY:
        print("\n🌟 GEMINI:")
        inicio = time.time()
        resposta = chamar_gemini(prompt)
        tempo = time.time() - inicio
        print(f"⏱️ Tempo: {tempo:.2f}s")
        print(f"📝 Resposta: {resposta[:200]}...\n")
    
    print("=" * 60)

# Teste
comparar_apis("Liste 5 temas de festa infantil populares em 2025")

## 💾 Exportar Resultados

In [ ]:
import pandas as pd
from datetime import datetime

# Exemplo: Gerar múltiplas descrições e salvar em Excel
temas = [
    {"tema": "Frozen", "idade": 5, "convidados": 30},
    {"tema": "Patrulha Canina", "idade": 4, "convidados": 25},
    {"tema": "Super-Heróis", "idade": 6, "convidados": 40},
    {"tema": "Unicórnio", "idade": 5, "convidados": 35},
    {"tema": "Dinossauros", "idade": 7, "convidados": 45},
]

resultados = []

print("🔄 Gerando descrições para múltiplos temas...\n")

for item in temas:
    print(f"⏳ Processando: {item['tema']}...")
    descricao = gerar_descricao_festa(
        tema=item['tema'],
        idade=item['idade'],
        convidados=item['convidados'],
        api="manus"
    )
    
    resultados.append({
        "Tema": item['tema'],
        "Idade": item['idade'],
        "Convidados": item['convidados'],
        "Descrição": descricao
    })

# Criar DataFrame
df = pd.DataFrame(resultados)

# Salvar em Excel
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"Festeja_Kids_Descricoes_IA_{timestamp}.xlsx"
df.to_excel(filename, index=False)

print(f"\n✅ Arquivo salvo: {filename}")
print(f"\n📥 Faça o download no menu 'Arquivos' do Colab\n")

# Mostrar preview
print("📋 Preview:")
print(df[['Tema', 'Idade', 'Convidados']].to_string(index=False))

# Download automático
from google.colab import files
files.download(filename)

## 🎯 Próximos Passos

1. **Teste diferentes APIs** e compare resultados
2. **Ajuste os prompts** para obter melhores respostas
3. **Integre no backend** do Festeja Kids
4. **Monitore custos** de cada API
5. **Crie workflows automatizados** combinando múltiplas APIs

---

## 📚 Recursos

- **Guia Completo**: `GUIA_INTEGRACAO_APIs.md`
- **Documentação Manus**: https://docs.manus.im
- **OpenAI Docs**: https://platform.openai.com/docs
- **Google AI Studio**: https://makersuite.google.com

---

**Desenvolvido para Festeja Kids 2.0**  
**Última Atualização:** 24 de novembro de 2025